In [ ]:
import torch
import numpy as np
import glob
import os
import re
import matplotlib.pyplot as plt

In [ ]:
from ipywidgets import *

In [ ]:
import pydicom

In [ ]:
from PIL import Image

In [ ]:
adr = '/media/data/rsna-miccai-brain-tumor-radiogenomic-classification/train/'

In [ ]:
pid = list(map(os.path.basename, glob.glob(os.path.join(adr, '*'))))

In [ ]:
flist = glob.glob(os.path.join(adr,'*/*/*'))
flist = [re.findall('train/(.*?)\/(.*?)\/Image-(.*)\.dcm', f)[0] for f in sorted(flist)]

In [ ]:
data = defaultdict(lambda : defaultdict(list))

In [ ]:
for pid, gname, imgid in flist:
    data[pid][gname].append(int(imgid)) 

In [ ]:
for f, v in data.items():
    for k, l in data[f].items():
        data[f][k].sort()
    

In [ ]:
pid , gname = '00000', 'FLAIR'
df = []
for imgid in data[pid][gname]:
    fname = os.path.join(adr,pid, gname, f'Image-{imgid}.dcm')
    img = pydicom.dcmread(fname).pixel_array
    df.append(img)

images = np.array(df)
mx = images.sum(axis=-1).sum(axis=-1)
alldata = images[np.where(mx>0)[0]]

In [ ]:
for name, item in data.items():
    length = max(item) - min(item) + 1
    if len(item) != length:
        print(name)

In [ ]:
dirs = ['FLAIR',  'T1w', 'T1wCE', 'T2w']

In [ ]:
from collections import defaultdict


In [ ]:
images = defaultdict()
for d in dirs:
    flist = glob.glob(os.path.join(adr, 'train','00000',d,'*.dcm'))
    m = [int(re.search("\d+(?=\.\w+$)", f).group(0)) for f in flist]
    print(len(m))
    files = [os.path.join(adr, 'train','00000',d,f'Image-{id}.dcm') for id in sorted(m)]
    df = []
    for id in sorted(m):
        fname = os.path.join(adr, 'train','00000',d,f'Image-{id}.dcm')
        df.append(pydicom.dcmread(fname).pixel_array)
    data = np.array(df)
    mx = data.max(axis=-1).max(axis=-1)
    images[d] = (65536 / mx[mx>0])[:,None,None] * data[mx > 0] 

In [ ]:
plt.plot(data[10].flatten())

In [ ]:
data = images[dirs[0]]
@interact
def update(idx:(0,len(data)-1)):
    im = Image.fromarray(data[idx].astype(np.uint16))
    display(im)

In [ ]:
data = images[dirs[3]]
@interact
def update(idx:(0,len(data)-1)):
    im = Image.fromarray(data[idx].astype(np.uint16))
    display(im)